```python
dim_x = 2
dim_y = 2
sigma = 0.1
n_comp = 16 
qkm_mle = QKMJointDenEstModel(dim_x=dim_x, dim_y=dim_y, sigma=sigma, n_comp=n_comp)
```

----------
```python
self.kernel_x = RBFKernelLayer(sigma, dim=dim_x, 
                                trainable=trainable_sigma,
                                min_sigma=min_sigma)
self.kernel_y = CosineKernelLayer()
self.kernel = CrossProductKernelLayer(dim1=dim_x, kernel1=self.kernel_x, kernel2=self.kernel_y)
self.qkmproj = QKMProjLayer(self.kernel,
                        dim_x=dim_x + dim_y,
                        n_comp=n_comp)

def call(self, inputs):
        log_probs = (tf.math.log(self.qkmproj(inputs) + 1e-12)
                        + self.kernel.log_weight())
        self.add_loss(-tf.reduce_mean(log_probs))
        return log_probs
```

### CrossProductKernelLayer

```python 
def call(self, A, B):
    '''
    Input:
        A: tensor of shape (bs, n, d)
        B: tensor of shape (m, d)
    Result:
        K: tensor of shape (bs, n, m)
    '''
    A1 = A[:, :, :self.dim1]
    A2 = A[:, :, self.dim1:]
    B1 = B[:, :self.dim1]
    B2 = B[:, self.dim1:]
    return self.kernel1(A1, B1) * self.kernel2(A2, B2)
```

Basicamente la idea es separar la entrada, para luego aplicar los dos kernels definidos, y su resultado multiplicarlo.
¿El producto de estos dos kernels por qué esta midiendo una similaridad?

### QKMProjLayer

```python
def call(self, inputs):
    comp_w = tf.abs(self.c_w) + 1e-10
    # normalize comp_w to sum to 1
    comp_w = comp_w / tf.reduce_sum(comp_w)
    in_v = inputs[:, tf.newaxis, :]
    out_vw = self.kernel(in_v, self.c_x) ** 2 # shape (b, 1, n_comp)
    out_w = tf.einsum('...j,...ij->...', comp_w, out_vw, optimize="optimal")
    return out_w
```

- Normalizar
- Agregar una dimensión
- Calcular un kernel
- Realizar producto para cada fila y aplanarlas en una suma

### Join_model

```python
self.kernel_x = RBFKernelLayer(sigma, dim=dim_x, 
                                trainable=trainable_sigma,
                                min_sigma=min_sigma)
self.kernel_y = CosineKernelLayer()
self.kernel = CrossProductKernelLayer(dim1=dim_x, kernel1=self.kernel_x, kernel2=self.kernel_y)
self.qkmproj = QKMProjLayer(self.kernel,
                        dim_x=dim_x + dim_y,
                        n_comp=n_comp)

def call(self, inputs):
        log_probs = (tf.math.log(self.qkmproj(inputs) + 1e-12)
                        + self.kernel.log_weight())
        self.add_loss(-tf.reduce_mean(log_probs))
        return log_probs
```

- Logaritmo al resultado que realizo el qkmP mas unos persos que guardo
- Valor negativo de la media y lo guarda